# Compute and compare different radial potentials

In [1]:
using Plots
using JAC

┌ Info: Precompiling JAC [830ae420-d14d-11e8-2f94-6b071437414d]
└ @ Base loading.jl:1342



Welcome to JAC:  A community approach to the computation of atomic structures, cascades and time evolutions [(C) Copyright by Stephan Fritzsche, Jena (2018-2021)].



We shall here describe how different atomic (radial) potentials can be calculated and applied within the JAC toolbox.

Any (radial) potential is handled in JAC by the data struct `Radial.Potential` that (may) contain a name, the radial function $Z(r)$ as well as the radial grid on which the potential is defined. The potential function `Zr` typically contains the effective charge $Z(r)$ as seen by an electron, so that the full radial potential is given by $V = -\:Z(r) / r$. This convention has been frequently applied in atomic physics and has also been adopted in JAC.


In [2]:
? Radial.Potential

`struct  Radial.Potential`  ... defines a struct for a local radial potential.

```
+ name           ::String            ... A name for the potential.
+ Zr             ::Array{Float64,1}  ... radial potential function Z(r) = - r * V(r) as usual in atomic structure theory.
+ grid           ::RadialGrid        ... radial grid on which the potential is defined.
```

---

`Radial.Potential()`  ... constructor to define an 'empty' instance of the radial potential.



Perhaps, the simplest choice of a radial potential is the **nuclear** potential which, of course, depends on the nuclear model. Let us first simply consider a $\;^{12}$C $\;$ carbon atom with a Fermi-distributed nuclear charge and a standard radial grid.


In [3]:
grid     = Radial.Grid(true)
nucModel = Nuclear.Model(6., "Fermi")

Fermi nuclear model for Z = 6.0 with mass = 12.18, radius R = 2.4934845703596404 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0.


Then, the corresponding nuclear potential is generated by simply calling:


In [4]:
nucPotential = Nuclear.nuclearPotential(nucModel, grid)

nuclear-potential: Fermi-distributed (radial) potential ... defined on 392 grid points ...
Zr:    [0.0, 0.015619818207380716, 0.035905804451892086, 0.060430845202356614, 0.08495453726302814, 0.10523736995823374, 0.11777879390742439, 0.12582486891780031, 0.14610064533238984, 0.17888509371708336  …  0..31112937694872056, 0.32313641267043025, 0.35185818903226784, 0.39828287685804126, 0.4543674906380097, 0.5103989493317181, 0.5566993479831148, 0.5853078885893802, 0.6022668932973185, 0.6428176395098482]  ...  [6.0, 6.000000000000002, 6.000000000000001, 6.000000000000002, 6.000000000000001, 6.0, 6.000000000000001, 6.0, 6.000000000000002, 6.000000000000002  …  6.000000000000002, 6.000000000000001, 6.000000000000002, 6.000000000000002, 6.000000000000001, 6.000000000000001, 6.0, 6.000000000000001, 6.000000000000001, 6.000000000000001]
Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints = 392,  ntL = 69,  ntS = 71, orderL = 7,  orderS = 8,  nsL = 62,  nsS = 63,  mesh = JAC.Radial.MeshGL(


As seen from the output, the effective charge Z(r) vanishes near to the origin but quickly increases to the full charge $Z = 6.\;$ at larger $r$-values, i.e. near to the surface and outside of the nucleus

To calculate **atomic** potentials, we also need to know the electron density around the nucleus. Most often, this density refers to a particular atomic level. Therefore, we first need to compute some of the low-lying levels of, for instance, neutral carbon. We shall here just compute the levels of the $1s^2 2s^2 2p^2$ ground-state configuration. We use the *simplest* approximation for these computation but will explain in some later tutorials how we could control and refine such simple structure calculations, if needed.


In [5]:
basis     = Basics.performSCF([Configuration("1s^2 2s^2 2p^2")], nucModel, grid, AsfSettings())
multiplet = Basics.performCI(basis, nucModel, grid, AsfSettings());


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^0 2p_3/2^2 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^1 2p_3/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 6.0 with mass = 12..18, radius R = 2.4934845703596404 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -1.80086298e+01    -1.80086350e+01    +2.86262559e-07    
      2      2s_1/2      -4.50269922e+00    -4.50269857e+00    -1.45559221e-07    
 


Iteration 1 for symmetries ... 
  1s_1/2::  en [a.u.] = -7.5200227e+00;   self-cons'cy = 4.1086e-01  [1.0000e+02 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -1.1797900e-02;   self-cons'cy = 9.9477e-01  [1.0000e+02 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -2.6580562e-01;   self-cons'cy = 8.8852e-01  [1.0000e+02 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -3.7295300e-01;   self-cons'cy = 8.4695e-01  [1.0000e+02 for sym-block kappa = -2]

Iteration 2 for symmetries ... 
  1s_1/2::  en [a.u.] = -1.0847252e+01;   self-cons'cy = 1.8115e-01  [1.0159e+00 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -9.2555310e-01;   self-cons'cy = 9.7483e-01  [1.0159e+00 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -8.0888174e-02;   self-cons'cy = 5.3337e-01  [1.0041e+00 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -9.3223634e-02;   self-cons'cy = 6.0005e-01  [1.0022e+00 for sym-block kappa = -2]

Iteration 3 for symmetries ... 
  1s_1/2::  en [a.u.] = -9.8287734e+00;   self-

  2p_3/2::  en [a.u.] = -1.5748592e-01;   self-cons'cy = 1.3057e-05  [2.9726e-04 for sym-block kappa = -2]

Iteration 22 for symmetries ... 
  1s_1/2::  en [a.u.] = -9.8875874e+00;   self-cons'cy = 3.3662e-07  [6.8162e-04 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -4.5750136e-01;   self-cons'cy = 3.0578e-06  [6.8162e-04 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -1.5780336e-01;   self-cons'cy = 5.9492e-06  [1.8607e-03 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -1.5748640e-01;   self-cons'cy = 1.5071e-06  [4.7217e-04 for sym-block kappa = -2]

Iteration 23 for symmetries ... 
  1s_1/2::  en [a.u.] = -9.8875828e+00;   self-cons'cy = 2.3042e-07  [1.0083e-03 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -4.5749959e-01;   self-cons'cy = 1.9347e-06  [1.0083e-03 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -1.5780278e-01;   self-cons'cy = 1.8294e-06  [8.3536e-03 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -1.5748782e-01;   self-cons'cy = 4.5043e-06  [1.0443e-


From this multiplet, we just take the *lowest* level (the ground level) in order to calculate the density for the (electronic) potential. In JAC, we have different choices for generating such an **electronic** potential that just arise from the electronic density, and as seen from ? compute at the REPL:

   •    ("radial potential: core-Hartree", grid::Radial.Grid, level::Level) 
       ... to compute a (radial) core-Hartree potential for the given level; 
           cf. Basics.computePotentialCoreHartree. A potential::RadialPotential 
           is returned. 

   •    ("radial potential: Kohn-Sham", grid::Radial.Grid, level::Level) 
       ... to compute a (radial) Kohn-Sham potential for the given level; 
           cf. Basics.computePotentialKohnSham. A potential::RadialPotential is 
           returned.

   •    ("radial potential: Dirac-Fock-Slater", grid::Radial..Grid, level::Level) 
       ... to compute a (radial) Dirac-Fock-Slater potential for the given level; this 
           potential is rather simple but includes some undesired self-interaction and 
           exhibits a wrong asymptotic behaviour. Cf. Basics.computePotentialDFS. 
           A potential::RadialPotential is returned.


In [6]:
level  = multiplet.levels[1]
potCH  = compute("radial potential: core-Hartree", grid, level)
potDFS = compute("radial potential: Dirac-Fock-Slater", grid, level)
potKS = compute("radial potential: Kohn-Sham", grid, level)

coreSubshells = Subshell[1s_1/2, 2s_1/2];    subshells = Subshell[1s_1/2, 2s_1/2, 2p_1/2, 2p_3/2]


Kohn-Sham (radial) potential ... defined on 392 grid points ...
Zr:    [0.0, 8.244855740993904e-5, 0.00010962587388256838, 8.749358831813382e-5, 5.097980506260815e-5, 2.4678201401699168e-5, 1.213322378215754e-5, 5.953426498752406e-6, -1.2660071531622308e-6, 6.913157671878871e-6  …  3.210653174499592e-5, 3.1010863590772863e-5, 2.6459560832301544e-5, 1.434455476562247e-5, -5.421543045549688e-6, -2.551102335625353e-5, -3.0287703356425806e-5, -3.0156511826126174e-5, -3.0663601046824346e-5, -3.386964581742926e-5]  ...  [-5.999099121737457, -5.999340240481848, -5.999518222456781, -5.999708604853091, -5.99908561295028, -5.998748469143674, -5.998751895785086, -5.999009014057045, -5.999295841291825, -5.999531060820585  …  -5.998970054118388, -5.999514364068934, -5.998790713254115, -5.998468381936907, -5.997932073994636, -5.997801855601406, -5.998912038612594, -5.998915289579009, -5.999124696426017, -5.999999999999995]
Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints = 392,  ntL = 69,


All these potentials are represented as *effective charge* $Z(r)$ and can be added to the nuclear potential, for example:

In [7]:
atomPotDFS = add(nucPotential, potDFS)

nuclear-potential: Fermi-distributed+DFS (radial) potential ... defined on 392 grid points ...
Zr:    [0.0, 0.015702266764790654, 0.03601543032577465, 0.060518338790674746, 0.08500551706809074, 0.10526204815963544, 0.11779092713120655, 0..12583082234429907, 0.14609937932523667, 0.17889200687475523  …  0.3111614834804656, 0.323167423534021, 0.35188464859310015, 0.3982972214128069, 0.4543620690949641, 0.5103734383083619, 0.5566690602797584, 0.5852777320775541, 0.6022362296962717, 0.6427837698640307]  ...  [0.0009008782625432588, 0.0006597595181538907, 0.0004817775432197635, 0.0002913951469105669, 0.0009143870497210216, 0.0012515308563258998, 0.00124810421491528, 0.0009909859429546586, 0.0007041587081770828, 0.00046893917941659424  …  0.0010299458816138696, 0.0004856359310672076, 0.0012092867458868284, 0.001531618063094875, 0.002067926005365095, 0.0021981443985952254, 0.0010879613874061889, 0.0010847104209918967, 0.0008753035739834658, 6.217248937900877e-15]
Radial grid:  rnt = 2.0e-6,  h


As seen from this output, the effective charge of the overall atomic potential goes to *zero* as required for a neutral atom. We can display these potentials also graphically by using `JAC.plot()`

In [11]:
nucPotential2 = Nuclear.nuclearPotential(Model(6.,"uniform"), grid)
nucPotential3 = Nuclear.nuclearPotential(Model(6., "point"), grid)
atomPotCH = add(nucPotential2, potCH)
atomPotKS = add(nucPotential3, potKS)
plot("radial potentials", [atomPotDFS,atomPotCH, atomPotKS], grid;N=300)

It is directly noticeable, that the functions based on the Kohn-Sham and Dirac-Fock-Slater potential correspond to the expectation for a neutral atom since the curve for large r approaches 0. In contrast, the curve for the Core-Hartree potential converges to a value greater than 0. That is due to the fact that the summation for the Core-Hartree potential runs only over all core orbitals as it is mainly applied for atoms with a single valence electron (alkali atoms). The graph thus shows the effective charge a single valence electron would "see". To illustrate the different nuclear potentials, we could also look at the functions at a closer angle, simply by choosing a smaller N (number of grid points to be considered).

In [13]:
plot("radial potentials", [atomPotDFS,atomPotCH, atomPotKS], grid;N=75)

The plot shows very well the caracteristics of the different nuclear models implemented in JAC.